<!-- ABSTRACT -->
Put this notebook in root directory!

In [ ]:
import json
import csv
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch import Tensor
from tensordict import TensorDict

from benchmarl.experiment import Experiment
from env.energy_trading import EnergyTradingEnv
from env.contract_proposal import ContractProposalEnv

from thesis_results.utils import set_plot_style, aid_mapping, color_mapping
set_plot_style()

In [ ]:
# Meta Madness
data = json.load(open("data/ausgrid/group_4.json", "r"))
agents = list(aid_mapping.keys())
test_days = data['meta']['test_days']
is_weekend_list = data['meta']['is_weekend']
is_summer_list = data['meta']['is_summer']
is_winter_list = data['meta']['is_winter']

seeds = range(8)
stage_2_seeds = range(8, 16)

# Get best model, prefer later epochs
def last_argmax(lst):
    arr = np.array(lst)
    return np.where(arr == arr.max())[0][-1]

### Learning Curves

In [ ]:
algos = ['DA', 'Pool', 'Mix', 'Grid', 'Mix_2',
         'MAPPO', 'MADDPG'] # 'MAPPO' and 'MADDPG' are just DA, Hack for Appendix B
learning_data = {algo: dict() for algo in algos}

# Parse data
for algo in ['DA', 'Pool', 'Mix', 'Grid', 'Mix_2']:

    train_data = []
    eval_data = []

    for seed in seeds:

        # Training
        filepath = f'results/{algo.lower().split("_")[0]}/{'stage_1/' if algo=='Mix' else 'stage_2/' if algo=='Mix_2' else ''}{'ippo' if algo=='Grid' else 'mappo'}/seed_{seed+8 if algo=='Mix_2' else seed}/seed_{seed+8 if algo=='Mix_2' else seed}/scalars/collection_agents_reward_episode_reward_mean.csv'
        
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            data = [float(row[1]) for row in reader]
            train_data.append(data)

        # Evaluation
        filepath = f'results/{algo.lower().split("_")[0]}/{'stage_1/' if algo=='Mix' else 'stage_2/' if algo=='Mix_2' else ''}{'ippo' if algo=='Grid' else 'mappo'}/seed_{seed+8 if algo=='Mix_2' else seed}/seed_{seed+8 if algo=='Mix_2' else seed}/scalars/eval_agents_reward_episode_reward_mean.csv'
        
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            data = [float(row[1]) for row in reader]
            eval_data.append(data)

    learning_data[algo]['train'] = np.array(train_data)
    learning_data[algo]['eval'] = np.array(eval_data)

In [ ]:
def plot_learning_curves(learning_data, algos, split="train", plot_var=True, save_path=None):
    """
    Plot learning curves (train or eval) for multiple algorithms.
    Rewards are converted to costs by multiplying with -8.

    Parameters
    ----------
    learning_data : dict
        Structure:
        learning_data[algo]['train'] -> (n_seeds, n_steps)
        learning_data[algo]['eval']  -> (n_seeds, n_steps)
    algos : list
        Algorithms to plot (keys in learning_data).
    split : str
        Which split to plot: 'train' or 'eval'.
    save_path : str or None
        Path to save figure if given.
    """

    assert split in ["train", "eval"], "split must be 'train' or 'eval'"

    plt.figure(figsize=(10, 6))

    steps = np.arange(learning_data[algos[0]][split].shape[1])

    for algo in algos:
        values = learning_data[algo][split] * -8  # reward → cost
        mean_vals = values.mean(axis=0)
        std_vals = values.std(axis=0)

        plt.plot(
            steps, mean_vals, 
            label=f"{algo}"
        )

        if plot_var:
            plt.fill_between(
                steps,
                mean_vals - std_vals,
                mean_vals + std_vals,
                alpha=0.2
            )

    plt.xlabel("Collection Steps")
    plt.ylabel("Episodic Community Cost ($)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
# Plot 5.1 and 5.2
plot_learning_curves(learning_data,
                     ['DA', 'Pool', 'Mix', 'Grid'],
                     split="eval",
                     save_path="thesis_results/plots/05_results/learning_curve_eval.png")

In [ ]:
# Hack for MADDPG, Appendix B
filepath = f'results/da/maddpg/seed_0/seed_0/scalars/collection_agents_reward_episode_reward_mean.csv'
with open(filepath, 'r') as f:
    reader = csv.reader(f)
    train_data = [float(row[1]) for row in reader]

filepath = f'results/da/maddpg/seed_0/seed_0/scalars/eval_agents_reward_episode_reward_mean.csv'
with open(filepath, 'r') as f:
    reader = csv.reader(f)
    eval_data = [float(row[1]) for row in reader]

learning_data['MAPPO']['train'] = learning_data['DA']['train']
learning_data['MAPPO']['eval'] = learning_data['DA']['eval']
learning_data['MADDPG']['train'] = np.array([train_data for _ in range(8)])
learning_data['MADDPG']['eval'] = np.array([eval_data for _ in range(8)])

plot_learning_curves(learning_data,
                     ['MAPPO', 'MADDPG'],
                     split="eval",
                     plot_var=False,
                     save_path="thesis_results/plots/appendix_B/maddpg_eval_curve.png")

In [ ]:
## Plot for Second Stage Training (5.3)

plt.figure(figsize=(10, 6))

steps = np.arange(16)

for split in ["train", "eval"]:
    values = learning_data["Mix_2"][split] * -8  # reward → cost
    mean_vals = values.mean(axis=0)
    std_vals = values.std(axis=0)

    plt.plot(
        steps, mean_vals, 
        label=f"{"Training" if split=="train" else "Evaluation"}"
    )
    plt.fill_between(
        steps,
        mean_vals - std_vals,
        mean_vals + std_vals,
        alpha=0.2
    )

plt.xlabel("Collection Steps")
plt.ylabel("Episodic Community Cost ($)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()

plt.show()

### Get Optimal Contracts

In [ ]:
# hashmap for optimal contracts
optimal_contracts = dict()

# Grid Config, Time of Use
ToU = [0.15, 0.22, 0.44]

FiT = 0.04  # Grid Config, Feed-in Tariff

# Timestep (hours) to ToU period, AER 2019
summer_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                        1, 1, 1, 1, 1, 1, 1, # 7 AM - 2 PM
                        2, 2, 2, 2, 2, 2, # 2 PM - 8 PM
                        1, 1, # 8 PM - 10 PM
                        0, 0] # 10 PM - 12 AM

winter_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # 7 AM - 5 PM
                        2, 2, 2, 2, # 5 PM - 9 PM
                        1, # 9 PM - 10 PM
                        0, 0] # 10 PM - 12 AM

weekend_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                   1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # 7 AM - 10 PM
                   0, 0] # 10 PM - 12 AM

def timestep_to_ToU_period(timestep, is_weekend, is_summer, is_winter):

    # Ultra realistic, AER 2019
    if is_weekend or (not (is_summer or is_winter)):
        return weekend_timemap[timestep % 24]
    elif is_summer:
        return summer_timemap[timestep % 24]
    else:
        return winter_timemap[timestep % 24]

In [ ]:
for seed in tqdm(stage_2_seeds):

    # Evaluation
    filepath = f"results/mix/stage_2/mappo/seed_{seed}/seed_{seed}/scalars/eval_agents_reward_episode_reward_mean.csv"
    
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        data = [float(row[1]) for row in reader]

    best_idx = last_argmax(data)
    best_exp = Experiment.reload_from_file(f"results/mix/stage_2/mappo/seed_{seed}/checkpoints/checkpoint_{(best_idx+1)*4096}.pt")
    config = best_exp.task.config
    proposal_env = ContractProposalEnv(config, render_mode=None)
    
    for is_weekend in [True, False]:
        for is_summer in [True, False]:
            for is_winter in [True, False]:
            
                if is_summer and is_winter:
                    continue

                optimal_contract = [dict() for _ in range(24)]

                for timestep in range(24):

                    t = timestep/24

                    obs = [ToU[timestep_to_ToU_period(timestep, is_weekend, is_summer, is_winter)],
                        FiT,
                        np.sin(2 * np.pi * t),
                        np.cos(2 * np.pi * t),
                        int(is_weekend),
                        int(is_summer),
                        int(is_winter)]

                    stacked_obs = Tensor(np.array([obs for aid in best_exp.group_map['agents']], dtype=np.float32))
                    obs_tdict = TensorDict(agents=TensorDict(observation=stacked_obs)).to(best_exp.policy.device)

                    actions = best_exp.policy.forward(obs_tdict)['agents']['action'].detach().cpu().numpy()
                    action_dict = {aid: actions[i] for i, aid in enumerate(best_exp.group_map['agents'])}

                    for aid in best_exp.group_map['agents']:
                        optimal_contract[timestep][aid] = action_dict[aid][0]

                optimal_contracts[(seed, is_weekend, is_summer, is_winter)] = optimal_contract

### Compute Results on Test Set

In [ ]:
# All the metrics we want to log
logging_metrics = ['qnt', 'da_grid_qnt', 'da_p2p_qnt', 'mix_pool_qnt', 'es_charge',
                   'reward', 'soc_action', 'price_action',
                   'avg_clearing_price', 'da_unmatched']

# Apply base policy to trade, assuming single group
def trade_forward(base_exp, obs):

    stacked_obs = Tensor(np.array([obs[aid] for aid in base_exp.group_map['agents']], dtype=np.float32))
    obs_tdict = TensorDict(agents=TensorDict(observation=stacked_obs)).to(base_exp.policy.device)

    actions = base_exp.policy.forward(obs_tdict)['agents']['action'].detach().cpu().numpy()
    action_dict = {aid: actions[i] for i, aid in enumerate(base_exp.group_map['agents'])}

    # Clear memory
    del obs_tdict
    del actions
    
    return action_dict

In [ ]:
def get_results(base_exp, trading_env, seed, algo, use_contracts=False):

    total_info = {aid: {metric: [[] for _ in range(len(test_days))] for metric in logging_metrics} for aid in agents}

    for i, day in enumerate(sorted(test_days)):
        
        if not use_contracts:
            obs, infos = trading_env.reset(options={"day": day})
        else:
            obs, infos = trading_env.reset(options={"day": day, "contracts": optimal_contracts[(seed+8,
                                                                                                is_weekend_list[day],
                                                                                                is_summer_list[day],
                                                                                                is_winter_list[day])]})

        # Step environment for an episode
        for t in range(trading_env.eps_len):

            # Environment actor
            actions = trade_forward(base_exp,obs)

            obs, rewards, terminations, truncations, infos = trading_env.step(actions)

            for aid in agents:
                total_info[aid]["qnt"][i].append(infos[aid]["qnt"])
                total_info[aid]["da_grid_qnt"][i].append(infos[aid]["da_grid_qnt"])
                total_info[aid]["da_p2p_qnt"][i].append(infos[aid]["da_p2p_qnt"])
                total_info[aid]["mix_pool_qnt"][i].append(infos[aid]["mix_pool_qnt"])
                total_info[aid]["reward"][i].append(rewards[aid])
                total_info[aid]["es_charge"][i].append(infos[aid]["es_charge"])
                total_info[aid]["soc_action"][i].append(actions[aid][0 if algo in ['pool', 'grid'] else 1])
                total_info[aid]["price_action"][i].append(actions[aid][0] if algo in ['da', 'mix'] else None)
                total_info[aid]["avg_clearing_price"][i].append(infos[aid]["avg_clearing_price"])
                total_info[aid]["da_unmatched"][i].append(infos[aid]["da_unmatched"])

    return total_info

In [ ]:
def aggregate_results(algo):

    # Dummy for task config
    base_exp_path = f"results/{algo}/{'stage_1/' if algo=='mix' else ''}{'ippo' if algo=='grid' else 'mappo'}/seed_0/checkpoints/checkpoint_524288.pt"
    base_exp = Experiment.reload_from_file(base_exp_path)
    config = base_exp.task.config

    # Single env for all evaluations
    trading_env = EnergyTradingEnv(config, render_mode=None)
    trading_env.reset(seed=2025)

    grand_info = {aid: {metric: list() for metric in logging_metrics} for aid in agents}

    for seed in tqdm(seeds):

        # Evaluation
        filepath = f'results/{algo}/{'stage_1/' if algo=='mix' else ''}{'ippo' if algo=='grid' else 'mappo'}/seed_{seed}/seed_{seed}/scalars/eval_agents_reward_episode_reward_mean.csv'
        
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            data = [float(row[1]) for row in reader]

        best_idx = last_argmax(data)
        best_exp = Experiment.reload_from_file(f"results/{algo}/{'stage_1/' if algo=='mix' else ''}{'ippo' if algo=='grid' else 'mappo'}/seed_{seed}/checkpoints/checkpoint_{(best_idx+1)*8192}.pt")

        info = get_results(best_exp, trading_env, seed, algo, use_contracts=True if algo == 'mix' else False)
        
        for aid in agents:
            for metric in logging_metrics:
                grand_info[aid][metric].append(info[aid][metric])

    return grand_info

In [ ]:
#########################
#### Get all results ####
#########################

base_info = aggregate_results('grid') # Grid Only Baseline
da_info = aggregate_results('da')
pool_info = aggregate_results('pool')
mix_info = aggregate_results('mix')

### Chapter 5 Tables

In [ ]:
# Table 5.1 - Daily Rewards
for aid in sorted(agents):

    base_reward = np.array(base_info[aid]["reward"]).sum(axis=-1)
    da_reward = np.array(da_info[aid]["reward"]).sum(axis=-1)
    pool_reward = np.array(pool_info[aid]["reward"]).sum(axis=-1)
    mix_reward = np.array(mix_info[aid]["reward"]).sum(axis=-1)

    print(f"{aid} & ${base_reward.mean():.2f} \\pm {base_reward.std():.2f}$ [{np.percentile(base_reward, 5):.2f}] & ${da_reward.mean():.2f} \\pm {da_reward.std():.2f}$ [{np.percentile(da_reward, 5):.2f}] & ${pool_reward.mean():.2f} \\pm {pool_reward.std():.2f}$ [{np.percentile(pool_reward, 5):.2f}] & ${mix_reward.mean():.2f} \\pm {mix_reward.std():.2f}$ [{np.percentile(mix_reward, 5):.2f}] \\\\")

ec_base_reward = sum([np.array(base_info[aid]["reward"]) for aid in agents]).sum(axis=-1)
ec_da_reward = sum([np.array(da_info[aid]["reward"]) for aid in agents]).sum(axis=-1)
ec_pool_reward = sum([np.array(pool_info[aid]["reward"]) for aid in agents]).sum(axis=-1)
ec_mix_reward = sum([np.array(mix_info[aid]["reward"]) for aid in agents]).sum(axis=-1)

print(f"Community & ${ec_base_reward.mean():.2f} \\pm {ec_base_reward.std():.2f}$ [{np.percentile(ec_base_reward, 5):.2f}] & ${ec_da_reward.mean():.2f} \\pm {ec_da_reward.std():.2f}$ [{np.percentile(ec_da_reward, 5):.2f}] & ${ec_pool_reward.mean():.2f} \\pm {ec_pool_reward.std():.2f}$ [{np.percentile(ec_pool_reward, 5):.2f}] & ${ec_mix_reward.mean():.2f} \\pm {ec_mix_reward.std():.2f}$ [{np.percentile(ec_mix_reward, 5):.2f}] \\\\")

In [ ]:
# Table 5.2 - Grid Reliance

# DA
ec_qnt = sum([np.where(np.array(da_info[aid]["qnt"])>=0, np.array(da_info[aid]["qnt"]), 0) for aid in agents]).sum(axis=-1)
ec_grid_qnt = sum([np.where(np.array(da_info[aid]["da_grid_qnt"])>=0, np.array(da_info[aid]["da_grid_qnt"]), 0) for aid in agents]).sum(axis=-1)
grid_reliance = ec_grid_qnt/ec_qnt
print(f"DA & ${grid_reliance.mean():.2f} \\pm {grid_reliance.std():.2f}$ [{np.percentile(grid_reliance, 95):.2f}] \\\\")

# Pool
ec_pool_qnt = sum([np.array(pool_info[aid]["qnt"]) for aid in agents])
ec_pool_qnt = np.where(ec_pool_qnt>=0, ec_pool_qnt, 0).sum(axis=-1)
ec_qnt = sum([np.where(np.array(pool_info[aid]["qnt"])>=0, np.array(pool_info[aid]["qnt"]), 0) for aid in agents]).sum(axis=-1)
grid_reliance = ec_pool_qnt/ec_qnt
print(f"Pool & ${grid_reliance.mean():.2f} \\pm {grid_reliance.std():.2f}$ [{np.percentile(grid_reliance, 95):.2f}] \\\\")

# Mix
ec_grid_qnt = sum([np.where(np.array(mix_info[aid]["da_grid_qnt"])>=0, np.array(mix_info[aid]["da_grid_qnt"]), 0) for aid in agents]).sum(axis=-1)
ec_pool_qnt = sum([np.array(mix_info[aid]["mix_pool_qnt"]) for aid in agents])
ec_pool_qnt = np.where(ec_pool_qnt>=0, ec_pool_qnt, 0).sum(axis=-1)
ec_total_grid = ec_grid_qnt + ec_pool_qnt
ec_qnt = sum([np.where(np.array(mix_info[aid]["qnt"])>=0, np.array(mix_info[aid]["qnt"]), 0) for aid in agents]).sum(axis=-1)
grid_reliance = ec_total_grid/ec_qnt
print(f"Mix & ${grid_reliance.mean():.2f} \\pm {grid_reliance.std():.2f}$ [{np.percentile(grid_reliance, 95):.2f}] \\\\")

### Chapter 6: ES Schedule

In [ ]:
# Boolean masks for seasonalities
summer_bool = np.array(is_summer_list)[test_days]
winter_bool = np.array(is_winter_list)[test_days]
weekend_bool = np.array(is_weekend_list)[test_days]

weekend_filter = weekend_bool | (~ (summer_bool | winter_bool))
summer_filter = summer_bool & (~weekend_bool)
winter_filter = winter_bool & (~weekend_bool)

In [ ]:
def plot_stacked_power(data_dict, agents, color_mapping, save_path=None):
    """
    Stacked bar chart with positive and negative values separated.
    """
    hours = np.arange(24)
    plt.figure()

    # Initialize running totals
    pos_bottom = np.zeros(24)
    neg_bottom = np.zeros(24)

    for agent in agents:
        values = np.squeeze(data_dict[agent])
        
        # Split into positive and negative parts
        pos_vals = np.where(values > 0, values, 0)
        neg_vals = np.where(values < 0, values, 0)
        
        # Plot positives (stacked upwards)
        plt.bar(hours, pos_vals, bottom=pos_bottom,
                color=color_mapping.get(agent, None),
                label=agent, width=0.75)
        pos_bottom += pos_vals
        
        # Plot negatives (stacked downwards)
        plt.bar(hours, neg_vals, bottom=neg_bottom,
                color=color_mapping.get(agent, None),
                width=0.75)
        neg_bottom += neg_vals

    # Formatting
    plt.xticks(hours)
    plt.xlabel("Hour of Day ($t$)")
    plt.ylabel("Charge or Discharge (kWh)", labelpad=8)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend(ncol=2, fontsize=16, frameon=True, loc='lower left', bbox_to_anchor=(0.025, 0.06))
    plt.tight_layout()

    ymin, ymax = plt.ylim()
    plt.ylim(ymin * 1.05, ymax * 1.05)   # add 5% space

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

In [ ]:
# Plot 6.1 and 6.2
mean_es_charge = {aid: np.array(mix_info[aid]['es_charge'])[:,weekend_filter,:].mean(axis=(0,1)) for aid in agents}
plot_stacked_power(mean_es_charge, sorted(agents), color_mapping, save_path="thesis_results/plots/06_discussion/mix_es_charge_weekend.png")

In [ ]:
def plot_soc_actions(data_dict, agents, color_mapping, save_path=None):
    """
    Plot SoC actions for each agent as line plots over 24 hours.

    Args:
        data_dict: dict {agent: np.array of shape (24,) or (24,1)}
        agents: list of agent names to plot
        color_mapping: dict {agent: color string}
        legend_loc: legend location code (default=0 = best)
        save_path: optional path to save figure
    """
    hours = np.arange(24)

    plt.figure()

    for agent in agents:
        values = np.squeeze(data_dict[agent])  # shape (24,)
        color = color_mapping.get(agent, None)
        plt.plot(hours, values, label=agent, color=color, linewidth=2.4)

    # Formatting
    plt.xticks(hours, [f"{h}" for h in hours])  # 0–23 labels
    plt.xlabel("Hour of Day ($t$)")
    plt.ylabel("SoC Action")
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.legend(fontsize=17) # 15.5 for appendix
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
# Plot 6.3
mean_soc_action = {aid: np.array(mix_info[aid]['soc_action']).mean(axis=(0,1)) for aid in agents}
plot_soc_actions(mean_soc_action, sorted(agents), color_mapping, save_path="thesis_results/plots/06_discussion/mix_soc_actions.png")

In [ ]:
### Hack for Appendix B ###

# Dummy for task config
exp_path = f"results/da/maddpg/seed_0/checkpoints/checkpoint_434176.pt" # only seed and run
exp = Experiment.reload_from_file(exp_path)
config = exp.task.config

# Single env for all evaluations
trading_env = EnergyTradingEnv(config, render_mode=None)
trading_env.reset(seed=2025)

maddpg_info = get_results(exp, trading_env, 0, 'damaddpg', use_contracts=False)

In [ ]:
### Plots for Appendix B ###

# mean_es_charge = {aid: np.array(maddpg_info[aid]['es_charge']).mean(axis=0) for aid in agents}
# plot_stacked_power(mean_es_charge, sorted(agents), color_mapping, save_path="thesis_results/plots/appendix_B/maddpg_es_charge.png")

mean_soc_action = {aid: np.array(maddpg_info[aid]['soc_action']).mean(axis=0) for aid in agents}
plot_soc_actions(mean_soc_action, sorted(agents), color_mapping, save_path="thesis_results/plots/appendix_B/maddpg_soc_actions.png")

### Chapter 6: P2P Volume Analysis

In [ ]:
da_p2p_qnt = sum([np.where(np.array(da_info[aid]["da_p2p_qnt"])>=0, np.array(da_info[aid]["da_p2p_qnt"]), 0) for aid in agents])
p2p_da = da_p2p_qnt.mean(axis=(0,1))

ec_qnt_pos = sum([np.where(np.array(pool_info[aid]["qnt"])>=0, np.array(pool_info[aid]["qnt"]), 0) for aid in agents])
ec_qnt_neg = sum([np.where(np.array(pool_info[aid]["qnt"])<0, -np.array(pool_info[aid]["qnt"]), 0) for aid in agents])
p2p_pool = np.minimum(ec_qnt_pos, ec_qnt_neg).mean(axis=(0,1))

ec_qnt_pos = sum([np.where(np.array(mix_info[aid]["mix_pool_qnt"])>=0, np.array(mix_info[aid]["mix_pool_qnt"]), 0) for aid in agents])
ec_qnt_neg = sum([np.where(np.array(mix_info[aid]["mix_pool_qnt"])<0, -np.array(mix_info[aid]["mix_pool_qnt"]), 0) for aid in agents])
da_p2p_qnt = sum([np.where(np.array(mix_info[aid]["da_p2p_qnt"])>=0, np.array(mix_info[aid]["da_p2p_qnt"]), 0) for aid in agents])
p2p_mix = (np.minimum(ec_qnt_pos, ec_qnt_neg) + da_p2p_qnt).mean(axis=(0,1))

In [ ]:
# Plot as a bar plot, Figure 6.4
hours = np.arange(24)  # 24 hours in a day

plt.bar(hours-0.22, p2p_da, color='skyblue', edgecolor='grey', width=0.22, label='DA', align='center')
plt.bar(hours, p2p_pool, color='salmon', edgecolor='grey', width=0.22, label='Pool', align='center')
plt.bar(hours+0.22, p2p_mix, color='lightgreen', edgecolor='grey', width=0.22, label='Mix', align='center')
plt.legend()

# Formatting
plt.xticks(hours, [f"{h}" for h in hours])  # 0–23 labels
plt.xlabel("Hour of Day ($t$)")
plt.ylabel("P2P Volume (kWh)")
plt.grid(True, linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
# Verify that there's early sell off on weekends
sell_off = np.array(base_info["prosumer_1"]["qnt"])[:,weekend_filter,:].mean(axis=(0,1))
print(sell_off)

### Chapter 6: DA Matching

In [ ]:
# Check if pricing causes unmatched trades in DA
da_unmatched = np.array(mix_info["prosumer_1"]["da_unmatched"]).flatten()
unique, counts = np.unique(da_unmatched, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# Plot 6.6, Average Matching Price
mean_da_price = np.array(da_info["prosumer_1"]['avg_clearing_price'])
mean_da_price = np.where(mean_da_price==-1, np.nan, mean_da_price)
mean_da_price = np.nanmean(mean_da_price, axis=(0,1))

mean_mix_price = np.array(mix_info["prosumer_1"]['avg_clearing_price'])
mean_mix_price = np.where(mean_mix_price==-1, np.nan, mean_mix_price)
mean_mix_price = np.nanmean(mean_mix_price, axis=(0,1))
    

plt.figure()
hours = np.arange(24)
plt.plot(hours, mean_da_price, label="DA", linewidth=2)
plt.plot(hours, mean_mix_price, label="Mix", linewidth=2)

# Formatting
plt.xticks(hours, [f"{h}" for h in hours])  # 0–23 labels
plt.xlabel("Hour of Day ($t$)")
plt.ylabel("DA Matching Price")
plt.grid(True, which="both", linestyle="--", alpha=0.6)
plt.legend(frameon=True, loc='lower left', bbox_to_anchor=(0.05, 0.05))
plt.tight_layout()

plt.savefig("thesis_results/plots/06_discussion/da_price_match.png", bbox_inches="tight")
plt.show()

In [ ]:
# Plot 6.5, Price Actions
def plot_price_actions(data_dict, agents, color_mapping, save_path=None):
    
    hours = np.arange(24)

    plt.figure()

    for agent in agents:
        values = np.squeeze(data_dict[agent])  # shape (24,)
        color = color_mapping.get(agent, None)
        plt.plot(hours, values, label=agent, color=color, linewidth=2.4)

    # Formatting
    plt.xticks(hours, [f"{h}" for h in hours])  # 0–23 labels
    plt.xlabel("Hour of Day ($t$)")
    plt.ylabel("Price Action")
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.legend(ncol=2, fontsize=15, frameon=True, loc='lower left', bbox_to_anchor=(0.01, 0.03))
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
mean_price_action = {aid: np.array(mix_info[aid]['price_action']).mean(axis=(0,1)) for aid in agents}
plot_price_actions(mean_price_action, sorted(agents), color_mapping, save_path="thesis_results/plots/06_discussion/mix_price_actions.png")

In [ ]:
# DA Grid Reliance, Table 6.1

for aid in sorted(agents):
    
    qnt = np.where(np.array(da_info[aid]["qnt"])>=0, np.array(da_info[aid]["qnt"]), 0).sum(axis=-1)
    grid_qnt = np.where(np.array(da_info[aid]["da_grid_qnt"])>=0, np.array(da_info[aid]["da_grid_qnt"]), 0).sum(axis=-1)
    grid_reliance = np.divide(grid_qnt, qnt, out=np.zeros_like(grid_qnt, dtype=float), where=qnt!=0)
    print(f"{aid} & ${grid_reliance.mean():.2f} \\pm {grid_reliance.std():.2f}$ [{np.percentile(grid_reliance, 95):.2f}] \\\\")

### Chapter 6: Fairness

In [ ]:
# Table 6.2
relative_savings = {'da':[], 'pool':[], 'mix':[]}

for aid in sorted(agents):

    base_reward = np.array(base_info[aid]["reward"]).sum(axis=-1)
    da_reward = np.array(da_info[aid]["reward"]).sum(axis=-1)
    pool_reward = np.array(pool_info[aid]["reward"]).sum(axis=-1)
    mix_reward = np.array(mix_info[aid]["reward"]).sum(axis=-1)

    da_savings = da_reward - base_reward
    pool_savings = pool_reward - base_reward
    mix_savings = mix_reward - base_reward

    relative_savings['da'].append(da_savings.flatten())
    relative_savings['pool'].append(pool_savings.flatten())
    relative_savings['mix'].append(mix_savings.flatten())

    print(f"{aid} & ${da_savings.mean():.2f} \\pm {da_savings.std():.2f}$ [{np.percentile(da_savings, 5):.2f}] & ${pool_savings.mean():.2f} \\pm {pool_savings.std():.2f}$ [{np.percentile(pool_savings, 5):.2f}] & ${mix_savings.mean():.2f} \\pm {mix_savings.std():.2f}$ [{np.percentile(mix_savings, 5):.2f}] \\\\")

ec_base_reward = sum([np.array(base_info[aid]["reward"]) for aid in agents]).sum(axis=-1)
ec_da_savings = sum([np.array(da_info[aid]["reward"]) for aid in agents]).sum(axis=-1) - ec_base_reward
ec_pool_savings = sum([np.array(pool_info[aid]["reward"]) for aid in agents]).sum(axis=-1) - ec_base_reward
ec_mix_savings = sum([np.array(mix_info[aid]["reward"]) for aid in agents]).sum(axis=-1) - ec_base_reward

print(f"Community & ${ec_da_savings.mean():.2f} \\pm {ec_da_savings.std():.2f}$ [{np.percentile(ec_da_savings, 5):.2f}] & ${ec_pool_savings.mean():.2f} \\pm {ec_pool_savings.std():.2f}$ [{np.percentile(ec_pool_savings, 5):.2f}] & ${ec_mix_savings.mean():.2f} \\pm {ec_mix_savings.std():.2f}$ [{np.percentile(ec_mix_savings, 5):.2f}] \\\\")

In [ ]:
def gini_per_scenario(x):
    """
    Compute Gini coefficient for each scenario/day.
    Args:
        x: np.ndarray of shape (8, N)  # 8 agents × N scenarios
    Returns:
        np.ndarray of shape (N,) with Gini per scenario
    """
    x = np.asarray(x)
    n_scenarios = x.shape[1]
    gini_vals = np.zeros(n_scenarios)

    for i in range(n_scenarios):
        vals = np.sort(x[:, i])

        # # Shift to non-negative
        # # Doesn't work so well, Ignore negatives for now
        # if vals[0]<0:
        #     vals = vals - vals[0] + 1e-8

        if np.all(vals == 0):
            gini_vals[i] = 0.0
            continue
        n = len(vals)
        index = np.arange(1, n + 1)
        gini_vals[i] = (2 * np.sum(index * vals) / (n * np.sum(vals))) - (n + 1) / n

    return gini_vals

for key in relative_savings:
    relative_savings[key] = np.array(relative_savings[key])

## Check negatives
# negative_counts = {key: np.sum(relative_savings[key] < 0, axis=1) for key in relative_savings}
# print(negative_counts)

# Remove negative cases for Gini
for key in relative_savings:
    relative_savings[key] = relative_savings[key][:, ~np.any(relative_savings[key] < 0, axis=0)]

gini = {key: gini_per_scenario(relative_savings[key]) for key in relative_savings}

# Table 6.3
for key in gini:
    print(f"{key} & ${gini[key].mean():.3f} \\pm {gini[key].std():.3f}$ [{np.percentile(gini[key], 95):.3f}] \\\\")

### Chapter 6: Optimal Contracts Plots

In [ ]:
def plot_contracts(data_dict, agents, color_mapping, save_path=None):
    
    hours = np.arange(24)

    plt.figure()

    for agent in agents:
        values = np.squeeze(data_dict[agent])  # shape (24,)
        color = color_mapping.get(agent, None)
        plt.plot(hours, values, label=agent, color=color, linewidth=2.4)

    # Formatting
    plt.xticks(hours, [f"{h}" for h in hours])  # 0–23 labels
    plt.xlabel("Hour of Day ($t$)")
    plt.ylabel("Contract (Pool Share)")
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.legend(ncol=2, fontsize=15.5, frameon=True, loc='lower right', bbox_to_anchor=(0.955, 0.01))
    plt.tight_layout()
    
    ymin, ymax = plt.ylim()
    plt.ylim(ymin * 0.95, ymax * 1)   # add 5% space

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
# Changed name to avoid conflict
opt_contract = {aid: [] for aid in agents}

for seed in stage_2_seeds:

    for is_summer, is_winter, is_weekend in [(False, False, False), # Other
                                             (False, False, True), # Other
                                             (False, True, True), # Other
                                             (True, False, True), # Other
                                             (True, False, False), # Summer
                                             (False, True, False)]: # Winter

        contract = optimal_contracts[(seed, is_weekend, is_summer, is_winter)]

        for aid in agents:
            opt_contract[aid].append([contract[t][aid] for t in range(24)])

for aid in agents:
    opt_contract[aid] = np.array(opt_contract[aid]).mean(axis=0)

# Plot 6.7
plot_contracts(opt_contract, sorted(agents), color_mapping, save_path="thesis_results/plots/06_discussion/optimal_contracts.png")